<a href="https://colab.research.google.com/github/sturu1/github-first/blob/master/2020_06_29_%EC%98%A4%ED%9B%84%EC%88%98%EC%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution 알아보기 p139
1. 이미지에서 convolution이 어떻게 유용한가?
2. Gaussian 스무딩(흐릿한이미지), 부드러운 이미지 만들어 보기
3. Sobel 미분필터, X축, Y축 편미분, 윤곽선 검출해보기


* 이미지 편미분 원리


![대체 텍스트](https://mblogthumb-phinf.pstatic.net/20150916_85/roboholic84_1442364979771XXm8h_PNG/filters.png?type=w2)


*  최종적으로 얻는 그레디언트 벡터(x편미분, y편미분)


$\nabla f = {\frac{\partial f(x, y)}{\partial_x}}, {\frac{\partial I(x, y)}{\partial_y}}$


* 윤곽선

$|G| = \sqrt{G^2_x * g^2_y}$

이미지 데이터이기때문에 컨볼루션을 사용해서 압축하는것


#$\frac{1}{2}$

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf


#데이터를 가져와보자
fashion_mnist = tf.keras.datasets.fashion_mnist #데이터와 연결
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data() # 실제로 데이터 가져옴

#=================filter

#sobel filter for Gx
kernelX = [[-1, 0, 1],
           [-2, 0, 2],
           [-1, 0, 1]]
kernelX = np.array(kernelX) # convert to numpy
kernelX.shape

#sobel filter for Gy
kernelY = [[1, 2, 1],
           [0, 0, 0],
           [-1, -2, -1]]
kernelY = np.array(kernelY)
kernerY.shape

#============ functions
#정규화 min-max noralize
def normalize(x):
  return (x - np.min(x)) / (np.max(x) - np.min(x))

#이미지 차원 알려주는 함수
# 이미지 차원 nX,nY
def getDimension(x):
  # 입력 이미지 x는 2차원이라고 가정
  return x.shape[0], x.shape[i] #nY, nX


def convolution(kernel, img):
  kY, kX = getDimension(kernel) #3*3
  nY, nX = getDimension(img) #28*28
  
  templmg = np.zeros((nY, nX), dtype = 'float32')

  for j in range(1, nY-1):# for domain Y
    for i in range(1,nX-1):# for domain X
      #kernel dimension, iterator k, l
      for l in range(0, kY):#0~2, 3개
        for k in range(0, kX):#0~2, 3개
        #컨볼루션 수행해서 temlmg에 맵핑해주면 됨
          tempImg[i, j] += img[i + k-1, j+l-1]* kernel[k, l] #convolution 수행


  return tempImg

def normalize(x):
  return (x - np.min(x)) / (np.max(x)-np.min(x))

#============== main code
#이미지 하나 불러오기
npImg = train_X[0]
print(npImg.dtype, npImg.shape, np.max(npImg), np.min(npImg))

dX = convolution(kernelX, npImg)
#dX = normalize(dX)
#print(dX)

dY = convolution(kernelY, npImg)
#dY = normalize(dY)
#print(dY)

#magnitude
Mag = np.sqrt(dX*dX + dY*dY)

dX = dX *255.0 #xcale up
dX = dX.astype(np.uint16) #int casting

dY = dY *255.0 #xcale up
dY = dY.astype(np.uint16) #int casting

plt.imshow(dX, camp='gray')

4423680/4422102 [==============================] - 0s 0us/step
uint8 (28, 28) 255 0


NameError: ignored